### binarized_module 잘 선택하기

In [143]:
import torch
import torchvision
import torchvision.transforms as tr
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from pytorchtools import EarlyStopping
#from pytorchtools import EarlyStopping_acc
from torchsummary import summary as summary_
import datetime
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.autograd import Function
#from binarized_modules import  BinarizeLinear,BinarizeConv2d,Binarizebatch2d
from binarized_modules_nonbias import  BinarizeLinear,BinarizeConv2d,Binarizebatch2d
from torch.autograd import Variable

In [144]:
def preprocessing(person, motion):  # person, motion에 해당하는 image 불러옴
    DirectoryPath = 'C:/Users/hojung/Documents/Anaconda_python/data/class12/'
    count = 100
    image = np.zeros(shape=(count, rows, cols, 1))
    label = []
    date = '220133'
    file_name = '_stft.txt'
    cwt_data = pd.read_csv(
        DirectoryPath + date + "_" + str(person) + "_" + str(motion) + file_name)
    for i in range(0, 100):
        df = np.fromstring(cwt_data['pixels'][i], dtype=float, sep=' ')
        df = np.reshape(df, (rows, cols, 1))
        image[i] = df
        if motion == 0:
            label.append(3 * (person - 1) + 0)
        else:
            label.append(3 * (person - 1) + motion - 1)

    return image, label



# 시작과 끝 좌표는 scale한 후의 좌표를 기준으로 함
def preprocessing_resize_crop(image,start_row,end_row,start_col,end_col,row_scale,col_scale): 
    crop_image = image[:,0:image.shape[1]:row_scale,0:image.shape[2]:col_scale]
    crop_image = crop_image[:,start_row:end_row,start_col:end_col]
    return crop_image

# ratio비율로 각 data set을 합치고 순서도 섞음
def concatenate_n_div(image0, label0, image1, label1, image2, label2, image3, label3, image4, label4, image5, label5, image6, label6, image7, label7, image8, label8, image9, label9, image10, label10, image11, label11):
    count = 100
    train_ratio = 0.7
    val_ratio = 0.15
    test_ratio = 0.15  # 적용안됨

    x_train = np.concatenate(
        (image0[0:int(count*train_ratio)],
         image1[0:int(count*train_ratio)],
         image2[0:int(count*train_ratio)],
         image3[0:int(count*train_ratio)],
         image4[0:int(count*train_ratio)],
         image5[0:int(count*train_ratio)],
         image6[0:int(count*train_ratio)],
         image7[0:int(count*train_ratio)],
         image8[0:int(count*train_ratio)],
         image9[0:int(count*train_ratio)],
         image10[0:int(count*train_ratio)],
         image11[0:int(count*train_ratio)]))
    y_train = np.concatenate(
        (label0[0:int(count*train_ratio)],
         label1[0:int(count*train_ratio)],
         label2[0:int(count*train_ratio)],
         label3[0:int(count*train_ratio)],
         label4[0:int(count*train_ratio)],
         label5[0:int(count*train_ratio)],
         label6[0:int(count*train_ratio)],
         label7[0:int(count*train_ratio)],
         label8[0:int(count*train_ratio)],
         label9[0:int(count*train_ratio)],
         label10[0:int(count*train_ratio)],
         label11[0:int(count*train_ratio)]))
    x_val = np.concatenate((image0[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            image1[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            image2[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            image3[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            image4[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            image5[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            image6[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            image7[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            image8[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            image9[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            image10[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            image11[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)]))
    y_val = np.concatenate((label0[int(count*train_ratio):int(count*train_ratio + count*val_ratio)],
                            label1[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            label2[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            label3[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            label4[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            label5[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            label6[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            label7[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            label8[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            label9[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            label10[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)],
                            label11[int(count*train_ratio):int(count *
                                                              train_ratio + count*val_ratio)]))
    x_test = np.concatenate((image0[int(count*train_ratio + count*val_ratio): count],
                             image1[int(count*train_ratio +
                                        count*val_ratio): count],
                             image2[int(count*train_ratio +
                                        count*val_ratio): count],
                             image3[int(count*train_ratio +
                                        count*val_ratio): count],
                             image4[int(count*train_ratio +
                                        count*val_ratio): count],
                             image5[int(count*train_ratio +
                                        count*val_ratio): count],
                             image6[int(count*train_ratio +
                                        count*val_ratio): count],
                             image7[int(count*train_ratio + 
                                        count*val_ratio): count],
                             image8[int(count*train_ratio + 
                                        count*val_ratio): count],
                             image9[int(count*train_ratio + 
                                        count*val_ratio): count],
                             image10[int(count*train_ratio + 
                                        count*val_ratio): count],
                             image11[int(count*train_ratio + 
                                        count*val_ratio): count]))
    y_test = np.concatenate((label0[int(count*train_ratio + count*val_ratio): count],
                             label1[int(count*train_ratio +
                                        count*val_ratio): count],
                             label2[int(count*train_ratio +
                                        count*val_ratio): count],
                             label3[int(count*train_ratio +
                                        count*val_ratio): count],
                             label4[int(count*train_ratio +
                                        count*val_ratio): count],
                             label5[int(count*train_ratio +
                                        count*val_ratio): count],
                             label6[int(count*train_ratio +
                                        count*val_ratio): count],
                             label7[int(count*train_ratio + 
                                        count*val_ratio): count],
                             label8[int(count*train_ratio + 
                                        count*val_ratio): count],
                             label9[int(count*train_ratio + 
                                        count*val_ratio): count],
                             label10[int(count*train_ratio + 
                                        count*val_ratio): count],
                             label11[int(count*train_ratio + 
                                        count*val_ratio): count]))


    s = np.arange(x_train.shape[0])
    np.random.shuffle(s)
    x_train = x_train[s]
    y_train = y_train[s]

    s = np.arange(x_val.shape[0])
    np.random.shuffle(s)
    x_val = x_val[s]
    y_val = y_val[s]

    s = np.arange(x_test.shape[0])
    np.random.shuffle(s)
    x_test = x_test[s]
    y_test = y_test[s]

    return x_train, y_train, x_val, y_val, x_test, y_test

class TensorData(Dataset):
    
    def __init__(self, x_data, y_data):
        self.x_data = torch.FloatTensor(x_data)
        self.x_data = self.x_data.permute(0,3,1,2) # 이미지 개수, 채널 수, 이미지 너비, 높이
        self.y_data = torch.LongTensor(y_data)
        self.len = self.y_data.shape[0]
        
    def __getitem__(self, index):
        
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.len

In [145]:
class Net3(nn.Module): # NET3 bias 있는거

    def __init__(self):
        super(Net3,self).__init__()
        self.infl_ratio=7
        
            # BinarizeConv2d(입력 채널수, 출력 채널수, 커널 사이즈, 스트라이드, 패딩)
        self.conv_L1 = BinarizeConv2d(1, 16*self.infl_ratio,kernel_size=3,stride=1,padding='same')
        self.max_L1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bn_L1 = nn.BatchNorm2d(16*self.infl_ratio) # nn.BatchNorm2d(입출력 채널수)
        self.htan_L1 = nn.Hardtanh()
    

        self.conv_L2 = BinarizeConv2d(16*self.infl_ratio, 16*self.infl_ratio,kernel_size=3,stride=1)
        self.max_L2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bn_L2 = nn.BatchNorm2d(16*self.infl_ratio) # nn.BatchNorm2d(입출력 채널수)
        self.htan_L2 = nn.Hardtanh()
 
        self.conv_L3 = BinarizeConv2d(16*self.infl_ratio, 16*self.infl_ratio,kernel_size=3,stride=1)
        self.max_L3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bn_L3 = nn.BatchNorm2d(16*self.infl_ratio) # nn.BatchNorm2d(입출력 채널수)
        self.htan_L3 = nn.Hardtanh()


        self.fc1 = BinarizeLinear(16*self.infl_ratio*3*2,112)
        self.human = BinarizeLinear(112,12)
        #self.htan_L4 = nn.Hardtanh()
        #self.bn_L4 = nn.BatchNorm1d(128) # nn.BatchNorm2d(입출력 채널수)
    
                
                
    def forward(self, x):
        x1 = self.conv_L1(x)
        x1 = self.max_L1(x1)
        x1 = self.bn_L1(x1)
        x1 = self.htan_L1(x1)
        
        x1 = self.conv_L2(x1)
        x1 = self.max_L2(x1)
        x1 = self.bn_L2(x1)
        x1 = self.htan_L2(x1)

        x1 = self.conv_L3(x1)
        x1 = self.max_L3(x1)
        x1 = self.bn_L3(x1)
        x1 = self.htan_L3(x1)

        x1 = x1.view(-1,16*self.infl_ratio*3*2)
        x1 = self.fc1(x1) # Linear
        #x1 = self.bn_L4(x1) # BatchNorm
        #x1 = self.htan_L4(x1) # htan
        
        human = self.human(x1) # Linear
        


        
        return human

In [3]:
class Net5(nn.Module): # NET3 bias 없는거

    def __init__(self):
        super(Net5,self).__init__()
        self.infl_ratio=7
        
            # BinarizeConv2d(입력 채널수, 출력 채널수, 커널 사이즈, 스트라이드, 패딩)
        self.conv_L1 = BinarizeConv2d(1, 16*self.infl_ratio,kernel_size=3,stride=1,padding='same',bias = False)
        self.max_L1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bn_L1 = nn.BatchNorm2d(16*self.infl_ratio) # nn.BatchNorm2d(입출력 채널수)
        self.htan_L1 = nn.Hardtanh()
    

        self.conv_L2 = BinarizeConv2d(16*self.infl_ratio, 16*self.infl_ratio,kernel_size=3,stride=1, bias = False)
        self.max_L2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bn_L2 = nn.BatchNorm2d(16*self.infl_ratio) # nn.BatchNorm2d(입출력 채널수)
        self.htan_L2 = nn.Hardtanh()
 
        self.conv_L3 = BinarizeConv2d(16*self.infl_ratio, 16*self.infl_ratio,kernel_size=3,stride=1,bias = False)
        self.max_L3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bn_L3 = nn.BatchNorm2d(16*self.infl_ratio) # nn.BatchNorm2d(입출력 채널수)
        self.htan_L3 = nn.Hardtanh()


        self.fc1 = BinarizeLinear(16*self.infl_ratio*3*2,112,bias = False)
        self.human = BinarizeLinear(112,12, bias = False)
        self.htan_L4 = nn.Hardtanh()
        self.bn_L4 = nn.BatchNorm1d(112) # nn.BatchNorm2d(입출력 채널수)
    
                
                
    def forward(self, x):
        x1 = self.conv_L1(x)
        x1 = self.max_L1(x1)
        x1 = self.bn_L1(x1)
        x1 = self.htan_L1(x1)
        
        x1 = self.conv_L2(x1)
        x1 = self.max_L2(x1)
        x1 = self.bn_L2(x1)
        x1 = self.htan_L2(x1)

        x1 = self.conv_L3(x1)
        x1 = self.max_L3(x1)
        x1 = self.bn_L3(x1)
        x1 = self.htan_L3(x1)

        x1 = x1.view(-1,16*self.infl_ratio*3*2)
        x1 = self.fc1(x1) # Linear
        x1 = self.bn_L4(x1) # BatchNorm
        x1 = self.htan_L4(x1) # htan
        
        human = self.human(x1) # Linear
        


        
        return human

In [146]:
class Net4(nn.Module): # NET3 bias 없는거

    def __init__(self):
        super(Net4,self).__init__()
        self.infl_ratio=7
        
            # BinarizeConv2d(입력 채널수, 출력 채널수, 커널 사이즈, 스트라이드, 패딩)
        self.conv_L1 = BinarizeConv2d(1, 16*self.infl_ratio,kernel_size=3,stride=1,padding='same',bias = False)
        self.max_L1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bn_L1 = nn.BatchNorm2d(16*self.infl_ratio) # nn.BatchNorm2d(입출력 채널수)
        self.htan_L1 = nn.Hardtanh()
    

        self.conv_L2 = BinarizeConv2d(16*self.infl_ratio, 16*self.infl_ratio,kernel_size=3,stride=1, bias = False)
        self.max_L2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bn_L2 = nn.BatchNorm2d(16*self.infl_ratio) # nn.BatchNorm2d(입출력 채널수)
        self.htan_L2 = nn.Hardtanh()
 
        self.conv_L3 = BinarizeConv2d(16*self.infl_ratio, 16*self.infl_ratio,kernel_size=3,stride=1,bias = False)
        self.max_L3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bn_L3 = nn.BatchNorm2d(16*self.infl_ratio) # nn.BatchNorm2d(입출력 채널수)
        self.htan_L3 = nn.Hardtanh()


        self.fc1 = BinarizeLinear(16*self.infl_ratio*3*2,112,bias = False)
        self.human = BinarizeLinear(112,12, bias = False)
        #self.htan_L4 = nn.Hardtanh()
        #self.bn_L4 = nn.BatchNorm1d(128) # nn.BatchNorm2d(입출력 채널수)
    
                
                
    def forward(self, x):
        x1 = self.conv_L1(x)
        x1 = self.max_L1(x1)
        x1 = self.bn_L1(x1)
        x1 = self.htan_L1(x1)
        
        x1 = self.conv_L2(x1)
        x1 = self.max_L2(x1)
        x1 = self.bn_L2(x1)
        x1 = self.htan_L2(x1)

        x1 = self.conv_L3(x1)
        x1 = self.max_L3(x1)
        x1 = self.bn_L3(x1)
        x1 = self.htan_L3(x1)

        x1 = x1.view(-1,16*self.infl_ratio*3*2)
        x1 = self.fc1(x1) # Linear
        #x1 = self.bn_L4(x1) # BatchNorm
        #x1 = self.htan_L4(x1) # htan
        
        human = self.human(x1) # Linear
        


        
        return human

In [147]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cuda device


In [6]:
net = Net3().to(device)
summary_(net,(1,36,28),batch_size=32)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
    BinarizeConv2d-1          [32, 112, 36, 28]           1,120
         MaxPool2d-2          [32, 112, 18, 14]               0
       BatchNorm2d-3          [32, 112, 18, 14]             224
          Hardtanh-4          [32, 112, 18, 14]               0
    BinarizeConv2d-5          [32, 112, 16, 12]         113,008
         MaxPool2d-6            [32, 112, 8, 6]               0
       BatchNorm2d-7            [32, 112, 8, 6]             224
          Hardtanh-8            [32, 112, 8, 6]               0
    BinarizeConv2d-9            [32, 112, 6, 4]         113,008
        MaxPool2d-10            [32, 112, 3, 2]               0
      BatchNorm2d-11            [32, 112, 3, 2]             224
         Hardtanh-12            [32, 112, 3, 2]               0
   BinarizeLinear-13                  [32, 112]          75,376
   BinarizeLinear-14                   

In [148]:
def train(dataloader, model, loss_fn, optimizer):
    model.train()
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device), 

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred,y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        for p in list(model.parameters()):
            if hasattr(p,'org'):
                p.data.copy_(p.org)
        optimizer.step()
        for p in list(model.parameters()):
            if hasattr(p,'org'):
                p.org.copy_(p.data.clamp_(-1,1))
                
        if batch % 40 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            loss = loss_fn(pred,y)
            test_loss += loss.item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy_human: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    return 100*correct

In [149]:
# SCALE:16 /1초 추출 + 0.2초 씩 이동 / row 30
#=========================================================
#================파라미터 설정==========================
#============================================================
# STFT = (128, 29) //  CWT = (81,1920)
# 좌표는 스케일 이후 범위
start_row = 46
end_row = 82
scale_row = 1
rows = 128

start_col = 0
end_col = 28
scale_col = 1
cols = 29

aug = 1  # 1이면 augmentation 0이면 X

gen = ImageDataGenerator(
                         width_shift_range=0.2
                            )

# dataset load
image1, label1 = preprocessing(1, 0)
image2, label2 = preprocessing(1, 2)
image3, label3 = preprocessing(1, 3)
image4, label4 = preprocessing(2, 0)
image5, label5 = preprocessing(2, 2)
image6, label6 = preprocessing(2, 3)
image7, label7 = preprocessing(3, 0)
image8, label8 = preprocessing(3, 2)
image9, label9 = preprocessing(3, 3)
image10, label10 = preprocessing(4, 0)
image11, label11 = preprocessing(4, 2)
image12, label12 = preprocessing(4, 3)

# 정규화 추가
total_img = np.concatenate((image1, image2, image3, image4, image5, image6, image7,
 image8, image9, image10, image11, image12))
total_std = total_img.std()
total_mean = total_img.mean()



In [ ]:
## net1
epochs = 230  # 학습을 원하는 만큼 수정요
try_num = 10 # 새로 학습하는 횟수 => 정확도 평균 계산하기 위함
human_result_acc = np.zeros(try_num*2*epochs).reshape(try_num,2,epochs)
motion_result_acc = np.zeros(try_num*2*epochs).reshape(try_num,2,epochs)
average_result_acc = np.zeros(try_num*epochs).reshape(try_num,epochs)
average_acc = 0
saveis = False
start = datetime.datetime.now()
for a in range(try_num):

    model = Net4() # 모델 설정
    model.cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    
    s = np.arange(image1.shape[0])
    np.random.shuffle(s)
    image1_shuff = image1[s]

    s = np.arange(image2.shape[0])
    np.random.shuffle(s)
    image2_shuff = image2[s]

    s = np.arange(image3.shape[0])
    np.random.shuffle(s)
    image3_shuff = image3[s]

    s = np.arange(image4.shape[0])
    np.random.shuffle(s)
    image4_shuff = image4[s]

    s = np.arange(image5.shape[0])
    np.random.shuffle(s)
    image5_shuff = image5[s]

    s = np.arange(image6.shape[0])
    np.random.shuffle(s)
    image6_shuff = image6[s]

    s = np.arange(image7.shape[0])
    np.random.shuffle(s)
    image7_shuff = image7[s]

    s = np.arange(image8.shape[0])
    np.random.shuffle(s)
    image8_shuff = image8[s]

    s = np.arange(image9.shape[0])
    np.random.shuffle(s)
    image9_shuff = image9[s]

    s = np.arange(image10.shape[0])
    np.random.shuffle(s)
    image10_shuff = image10[s]

    s = np.arange(image11.shape[0])
    np.random.shuffle(s)
    image11_shuff = image11[s]

    s = np.arange(image12.shape[0])
    np.random.shuffle(s)
    image12_shuff = image12[s]

    # 크기에 맞게 자름
    image1_crop = preprocessing_resize_crop(
        image1_shuff, start_row, end_row, start_col, end_col, scale_row, scale_col)
    image2_crop = preprocessing_resize_crop(
        image2_shuff, start_row, end_row, start_col, end_col, scale_row, scale_col)
    image3_crop = preprocessing_resize_crop(
        image3_shuff, start_row, end_row, start_col, end_col, scale_row, scale_col)
    image4_crop = preprocessing_resize_crop(
        image4_shuff, start_row, end_row, start_col, end_col, scale_row, scale_col)
    image5_crop = preprocessing_resize_crop(
        image5_shuff, start_row, end_row, start_col, end_col, scale_row, scale_col)
    image6_crop = preprocessing_resize_crop(
        image6_shuff, start_row, end_row, start_col, end_col, scale_row, scale_col)
    image7_crop = preprocessing_resize_crop(
        image7_shuff, start_row, end_row, start_col, end_col, scale_row, scale_col)
    image8_crop = preprocessing_resize_crop(
        image8_shuff, start_row, end_row, start_col, end_col, scale_row, scale_col)
    image9_crop = preprocessing_resize_crop(
        image9_shuff, start_row, end_row, start_col, end_col, scale_row, scale_col)
    image10_crop = preprocessing_resize_crop(
        image10_shuff, start_row, end_row, start_col, end_col, scale_row, scale_col)
    image11_crop = preprocessing_resize_crop(
        image11_shuff, start_row, end_row, start_col, end_col, scale_row, scale_col)
    image12_crop = preprocessing_resize_crop(
        image12_shuff, start_row, end_row, start_col, end_col, scale_row, scale_col)

    # 자른 image를 각 data set으로 나눠서 합침
    x_train, y_train, x_val, y_val, x_test, y_test = concatenate_n_div(
        image1_crop, label1, image2_crop, label2, image3_crop, label3, image4_crop, label4, image5_crop, 
        label5, image6_crop, label6, image7_crop, label7, image8_crop, label8, image9_crop, label9, image10_crop, 
        label10, image11_crop, label11, image12_crop, label12)


    x_train = (x_train - total_mean) / total_std
    x_val = (x_val - total_mean) / total_std
    x_test = (x_test - total_mean) / total_std

    # 보강할 학습데이터 이미지 생성
    if aug == 1:
        augment_ratio = 9   # 전체 데이터의 150%
        augment_size = int(augment_ratio * x_train.shape[0])

        # 전체 x_train 개수의 150% 비율만큼
        randidx = np.random.randint(x_train.shape[0], size=augment_size)

        # 임의로 선택된 데이터는 원본데이터를 참조하기 때문에
        # 원본데이터에 영향을 줄수 있음. 그래서 copy() 함수를 통해 안전하게 복사본 만듬
        x_augmented = x_train[randidx].copy()  
        y_augmented = y_train[randidx].copy()

        #  이미지 보강 실행
        x_augmented, y_augmented = gen.flow(x_augmented, y_augmented, 
                                            batch_size=augment_size,
                                            shuffle=False).next()

        x_train = np.concatenate((x_train,x_augmented))
        y_train = np.concatenate((y_train,y_augmented))
        s = np.arange(x_train.shape[0])
        np.random.shuffle(s)
        x_train = x_train[s]
        y_train = y_train[s]

    train_data = TensorData(x_train,y_train)
    test_data = TensorData(x_test,y_test)
    valid_data = TensorData(x_val,y_val)
    train_loader = DataLoader(train_data, batch_size = 32, shuffle =True)
    test_loader = DataLoader(test_data, batch_size = 32, shuffle =True)
    valid_loader = DataLoader(valid_data, batch_size = 32, shuffle =True)
    for t in range(epochs):
        print(f"total try_num {a+1}, Epoch {t+1}\n-------------------------------")
        train(train_loader, model, criterion, optimizer)

        human_result_acc[a][0][t] = test(valid_loader, model, criterion)
        human_result_acc[a][1][t] = test(test_loader, model, criterion)
        #if average_result_acc.max() < ((human_result_acc[a][0][t] + human_result_acc[a][1][t]) / 2) and best_acc <= ((human_result_acc[a][0][t] + human_result_acc[a][1][t]) / 2):
        if average_result_acc.max() < ((human_result_acc[a][0][t] + human_result_acc[a][1][t]) / 2):
            #save_name = './save/save_point' + str(a)+str(t)+'.pt'
            #torch.save(model.state_dict(), save_name)
           #save_test_loader = DataLoader(test_data, batch_size = 32, shuffle =True)
            #save_valid_loader = DataLoader(valid_data, batch_size = 32, shuffle =True)
            torch.save(model.state_dict(), './save_best/final/model_point8.pt')
            torch.save(test_loader, './save_best/final/test_loader_point8.pt')
            torch.save(valid_loader, './save_best/final/valid_loader_point8.pt')
            print(f"try : {a+1} epoch : {t+1} save model and data\n")
            saveis = True
        average_result_acc[a][t] = (human_result_acc[a][0][t] + human_result_acc[a][1][t]) / 2
print("Done!")
end = datetime.datetime.now()
print('동작시간 :', end - start)
for b in range(try_num):
    print(f"try : {b+1} // epoch : {np.argmax(average_result_acc[b])+1} // Max_average_accuracy = {max(average_result_acc[b]):0.1f},\
 Max_accuray_valid_human = {human_result_acc[b][0][np.argmax(average_result_acc[b])]:0.1f},\
 Max_accuray_test_human = {human_result_acc[b][1][np.argmax(average_result_acc[b])]:0.1f}\n ")
    average_acc += max(average_result_acc[b])
average_acc /= try_num
print(f"total average_acc = {average_acc:0.1f}")
print("Save is :", saveis,"\n")

### savepoint8

In [153]:
print('동작시간 :', end - start)
print("Save is :", saveis,"\n")
for b in range(try_num):
    print(f"try : {b+1} // epoch : {np.argmax(average_result_acc[b])+1} // Max_average_accuracy = {max(average_result_acc[b]):0.1f},\
 Max_accuray_valid_human = {human_result_acc[b][0][np.argmax(average_result_acc[b])]:0.1f},\
 Max_accuray_test_human = {human_result_acc[b][1][np.argmax(average_result_acc[b])]:0.1f}\n ")

print(f"total average_acc = {average_acc:0.1f}")

동작시간 : 0:53:14.133139
Save is : True 

try : 1 // epoch : 188 // Max_average_accuracy = 86.4, Max_accuray_valid_human = 85.0, Max_accuray_test_human = 87.8
 
try : 2 // epoch : 160 // Max_average_accuracy = 86.9, Max_accuray_valid_human = 85.6, Max_accuray_test_human = 88.3
 
try : 3 // epoch : 213 // Max_average_accuracy = 86.9, Max_accuray_valid_human = 86.7, Max_accuray_test_human = 87.2
 
try : 4 // epoch : 114 // Max_average_accuracy = 87.8, Max_accuray_valid_human = 88.3, Max_accuray_test_human = 87.2
 
try : 5 // epoch : 78 // Max_average_accuracy = 88.9, Max_accuray_valid_human = 87.2, Max_accuray_test_human = 90.6
 
try : 6 // epoch : 60 // Max_average_accuracy = 86.1, Max_accuray_valid_human = 85.6, Max_accuray_test_human = 86.7
 
try : 7 // epoch : 140 // Max_average_accuracy = 86.7, Max_accuray_valid_human = 88.3, Max_accuray_test_human = 85.0
 
try : 8 // epoch : 221 // Max_average_accuracy = 86.7, Max_accuray_valid_human = 87.8, Max_accuray_test_human = 85.6
 
try : 9 // 

### savepoint7

In [151]:
print('동작시간 :', end - start)
print("Save is :", saveis,"\n")
for b in range(try_num):
    print(f"try : {b+1} // epoch : {np.argmax(average_result_acc[b])+1} // Max_average_accuracy = {max(average_result_acc[b]):0.1f},\
 Max_accuray_valid_human = {human_result_acc[b][0][np.argmax(average_result_acc[b])]:0.1f},\
 Max_accuray_test_human = {human_result_acc[b][1][np.argmax(average_result_acc[b])]:0.1f}\n ")

print(f"total average_acc = {average_acc:0.1f}")

동작시간 : 0:49:39.196803
Save is : True 

try : 1 // epoch : 185 // Max_average_accuracy = 86.4, Max_accuray_valid_human = 82.2, Max_accuray_test_human = 90.6
 
try : 2 // epoch : 88 // Max_average_accuracy = 87.8, Max_accuray_valid_human = 88.9, Max_accuray_test_human = 86.7
 
try : 3 // epoch : 209 // Max_average_accuracy = 86.1, Max_accuray_valid_human = 86.1, Max_accuray_test_human = 86.1
 
try : 4 // epoch : 168 // Max_average_accuracy = 88.1, Max_accuray_valid_human = 92.2, Max_accuray_test_human = 83.9
 
try : 5 // epoch : 97 // Max_average_accuracy = 86.9, Max_accuray_valid_human = 85.0, Max_accuray_test_human = 88.9
 
try : 6 // epoch : 161 // Max_average_accuracy = 87.5, Max_accuray_valid_human = 92.8, Max_accuray_test_human = 82.2
 
try : 7 // epoch : 136 // Max_average_accuracy = 87.2, Max_accuray_valid_human = 84.4, Max_accuray_test_human = 90.0
 
try : 8 // epoch : 225 // Max_average_accuracy = 87.5, Max_accuray_valid_human = 86.7, Max_accuray_test_human = 88.3
 
try : 9 // 

### Load Model

In [156]:
best_model = Net4()
best_model.to(device)
best_model.eval()
best_model.load_state_dict(torch.load('./save_best/final/model_point7.pt'))
best_test_loader = torch.load('./save_best/final/test_loader_point7.pt')
best_valid_loader = torch.load('./save_best/final/valid_loader_point7.pt')
criterion1 = nn.CrossEntropyLoss()
best_vaild_acc = test(best_valid_loader, best_model, criterion1)
best_test_acc = test(best_test_loader, best_model, criterion1)
best_acc = (best_vaild_acc + best_test_acc) / 2
print(f"Best Accuracy is {best_acc:0.1f}")

Test Error: 
 Accuracy_human: 89.4%, Avg loss: 0.055516 

Test Error: 
 Accuracy_human: 90.0%, Avg loss: 0.056125 

Best Accuracy is 89.7


### threshold extraction

In [9]:
def binary2hex(num): # convert binary to hex
  hexstr = f'{int(num, 2):X}'
  return hexstr

### Verilog MIF 파일 생성을 위한 것으로 주소 및 값을 출력

In [157]:
# Conv1 weight
conv1 = best_model.conv_L1.weight.data
conv1_weight = ['','','','','','','','','']
for i in range(112):
  for row in range(3):
    for col in range(3):
      if conv1[i][0][row][col] == -1:
        conv1_weight[(row*3)+col] = '0' + conv1_weight[(row*3)+col]
      else:
        conv1_weight[(row*3)+col] = '1' + conv1_weight[(row*3)+col]

conv1_weight_hex = ['','','','','','','','','']
for i in range(9):
    conv1_weight_hex[i] = binary2hex(conv1_weight[i])
for i in range(9):
    print(i,"     :   ",conv1_weight_hex[i]+";")

0      :    A96B34E17E4B1497DB803174B6C7;
1      :    7765C6665ECFFB7B29912CA61E43;
2      :    E160BB6368FF9B2D72A5DA180D4A;
3      :    F6E0AF78472E46B2C32689B4D132;
4      :    9363F0966B4D5673391FBFD67F24;
5      :    9638838B215253B51AFB6B5E8EFD;
6      :    7BE2FB107179B966A6CAEEA265FB;
7      :    EFA38AF68B3D9B0A4C59969BDE4F;
8      :    DCA3F9A5AC4FE3E4E8274F064B89;


In [158]:
# Conv2 weight
conv2 = best_model.conv_L2.weight.data
conv2_weight = ['']
for i in range(1007):
    conv2_weight.append('')

for i in range(112):
  for j in range(112):
    for row in range(3):
      for col in range(3):
        if conv2[i][j][row][col] == -1:
          conv2_weight[(row*3)+col+(9*i)] = '0' + conv2_weight[(row*3)+col+(9*i)]
        else:
          conv2_weight[(row*3)+col+(9*i)] = '1' + conv2_weight[(row*3)+col+(9*i)]


conv2_weight_hex = ['']
for i in range(1007):
    conv2_weight_hex.append('')

for i in range(1008):
    conv2_weight_hex[i] = binary2hex(conv2_weight[i])

for i in range(1008):
    print(i+9,"     :   ", conv2_weight_hex[i]+";")


9      :    38BCF1EBB5C6B8EDFDB929D9AFBB;
10      :    AE3BE765545DB4BF15DD3C7EAF9F;
11      :    EA602644B8FFBFA9989410588A71;
12      :    FC9F1FBE3A767B7CDBBBFE5FD34B;
13      :    3CA95E9D3D246E4327CFA6E7E42D;
14      :    AAF26C7A94137F019D7F124F3881;
15      :    FEB2BAB3493B0711A4ADCD1FED22;
16      :    6DDE54788AC6EC9EADCC636FE5EE;
17      :    79DC0D60D1B3AA1897FEEB61ACF6;
18      :    2259A001A10D3B4A050BF0868FE7;
19      :    3CA55A860371EA22DA92EB20AC;
20      :    16D861F140C132DBA1212291B80;
21      :    D4F7B677FB1FFFB71DDE598D6DA;
22      :    B88AD1EDACFFABF7E2D847428CAB;
23      :    DAE3DC9D1D3FD6EAF37C9E92E6F8;
24      :    965740E475D2FEEB5898DFDCDAFC;
25      :    16D1041A47A4BFBB3A6F00DFAC24;
26      :    B72BCEF66A3FCAF37BD9AD8EBEB5;
27      :    3DF6A53935BE4EA8B92A90FCE785;
28      :    D365914161080238301218D0DB24;
29      :    2E4322B029215379D9593F56EE25;
30      :    A3888BA99C201DA22040182D7020;
31      :    81648117D0B0B81A3200D52130C0;
32      :    316

In [159]:
# Conv3 weight
conv3 = best_model.conv_L3.weight.data
conv3_weight = ['']
for i in range(1007):
  conv3_weight.append('')

for i in range(112):
  for j in range(112):
    for row in range(3):
      for col in range(3):
        if conv3[i][j][row][col] == -1:
          conv3_weight[(row*3)+col+(9*i)] = '0' + conv3_weight[(row*3)+col+(9*i)]
        else:
          conv3_weight[(row*3)+col+(9*i)] = '1' + conv3_weight[(row*3)+col+(9*i)]

conv3_weight_hex = ['']
for i in range(1007):
    conv3_weight_hex.append('')

for i in range(1008):
  conv3_weight_hex[i] = binary2hex(conv3_weight[i])

for i in range(1008):
    print(i+1017,"     :   ", conv3_weight_hex[i]+";")

1017      :    26F84848B90727B81B8A9B946551;
1018      :    CF03180EF23F3F625F3F5D316B70;
1019      :    4F037AD7F6BA9A21D4ED88D2080C;
1020      :    6FFDB06BC37FBDD2C5FAC72A18D2;
1021      :    852F3A7BB3D63E4C27ECAE561D9C;
1022      :    594CA172CB1D1A2C203E419D23B9;
1023      :    106DA597C96C4ADE234481DE0BAD;
1024      :    850B0329BD924AA911D051C58A49;
1025      :    4148A42134A403B35E5459294391;
1026      :    C3D091EE8F6374B1D412F64D547;
1027      :    DC3DB812E835FE48448FA47F3D6B;
1028      :    490D1272E2A46B68945964533351;
1029      :    660F136EFCB6982815E1F3C59A7B;
1030      :    2E1F3328B5A42A000FD4FB39EF61;
1031      :    4A4C090380900248943E9D5AEF39;
1032      :    A26D2661C3BAFAFCE30E9C5B964D;
1033      :    F3EC7EF93BBACC5F73F78D0FE614;
1034      :    E22A18691B9882300814FF4377B3;
1035      :    1E4B42D74DC8454A20FAA6AAC0C7;
1036      :    1C13C8CB890925F2E0FB68E028B6;
1037      :    44F1E0C00A4E724AB4386484626C;
1038      :    6FB838BE02C972C4FF236A21B899;
1039      :

In [160]:
# FCL1 weight
fcl1 = best_model.fc1.weight.data

fcl1_weight = ['']
for i in range(671):
  fcl1_weight.append('')

for i in range(112):
  for j in range(6):
    for row in range(112):
       if fcl1[i][6*row+j] == -1:
         fcl1_weight[(i*6)+j] = '0' + fcl1_weight[(i*6)+j]
       else:
         fcl1_weight[(i*6)+j] = '1' + fcl1_weight[(i*6)+j]

fcl1_weight_hex = ['']
for i in range(671):
    fcl1_weight_hex.append('')

for i in range(672):
    fcl1_weight_hex[i] = binary2hex(fcl1_weight[i])

for i in range(672):
    print(i+2025,"     :   ",fcl1_weight_hex[i]+";")

2025      :    10AC312B5F5C80F54E8146048339;
2026      :    503430E0831C901004E5164EB065;
2027      :    B5933FF478A3C1737251A3831AB2;
2028      :    2DB21E12FB8370EEB9DB279A43B3;
2029      :    BB2D10B4052478038E6163CA3188;
2030      :    102F232E15553C0302A1718E25C4;
2031      :    231B5E86FE2360DA3212AFB03A54;
2032      :    86DE7408BBD200599FB4D87577F0;
2033      :    9AC5A5ED2B9ED3579D7A7A7649A1;
2034      :    CDE4BD7328DEF3DFFF773B3ECA85;
2035      :    E566F36A355EFF27C5E788BBDEE1;
2036      :    66F2D9FB74CEDD8AC7C7A989C7E8;
2037      :    5CFEE95D917D960106C12C8DB756;
2038      :    3465707CDD5CB7252E032DCF26C4;
2039      :    F3B31A14E8A5205AA337D5BE1FE3;
2040      :    A30A9A940E2A2C50AF12C0FB03C2;
2041      :    90B5605C8794413A2C2857C6F5C9;
2042      :    109DE02F4715D227246234C655C4;
2043      :    2574792086BD725784A7C748A31F;
2044      :    402C3A2086B431D3E850D4CC262D;
2045      :    A711B7C1EAA391DA34D8A62F32B5;
2046      :    360195B07BC1B4D69778267BC0BA;
2047      

In [161]:
# FCL2 weight
fcl2 = best_model.human.weight.data

fcl2_weight = ['']
for i in range(11):
    fcl2_weight.append('')

for i in range(12):
    for row in range(112):
       if fcl2[i][row] == -1:
         fcl2_weight[i] = '0' + fcl2_weight[i]
       else:
         fcl2_weight[i] = '1' + fcl2_weight[i]

fcl2_weight_hex = ['']
for i in range(11):
  fcl2_weight_hex.append('')

for i in range(12):
    fcl2_weight_hex[i] = binary2hex(fcl2_weight[i])
  
for i in range(12):
    print(i+2697,"     :   ",fcl2_weight_hex[i]+";")

2697      :    5BA53EFAB08C86D252D7FBE57789;
2698      :    C0E46C71874576546EBB2444E78F;
2699      :    B8991DF46CFA1D31929443BB8054;
2700      :    19037EBAA83212B0C393D268F458;
2701      :    808C8A5BD60154DC5DD3A560C6C2;
2702      :    B80F2D6069B03201710603FB98D4;
2703      :    99CAAB0F04D181788DE849B032BA;
2704      :    684551ECEB04B7D5D620A0580B2;
2705      :    B06E51150EC396657208772B915D;
2706      :    DF9F0ADFAC9B339945EA5AB5703A;
2707      :    845655912720C21D4A682C07098D;
2708      :    F52D070169DB1F44B0841B6BD24D;


In [10]:
def deci2hex(num): # convert decimal to hex
  hexstr = f'{int(num, 10):X}'
  return hexstr

### Convert Batchnorm weight to threshold

In [162]:
# Batchnorm 1
bn1_weight = best_model.bn_L1.weight.data
bn1_weight = bn1_weight.cpu()
bn1_weight = bn1_weight.numpy()

bn1_bias = best_model.bn_L1.bias.data
bn1_bias = bn1_bias.cpu()
bn1_bias = bn1_bias.numpy()

bn1_run_mean = best_model.bn_L1.running_mean.data
bn1_run_mean = bn1_run_mean.cpu()
bn1_run_mean = bn1_run_mean.numpy()

bn1_run_var = best_model.bn_L1.running_var.data
bn1_run_var = bn1_run_var.cpu()
bn1_run_var = bn1_run_var.numpy()

L1_threshold = -1*(bn1_bias*np.sqrt(bn1_run_var)/bn1_weight) + bn1_run_mean

L1_threshold_ceil = np.ceil(L1_threshold)

L1_threshold_ceil_hex = ['']
for i in range(111):
  L1_threshold_ceil_hex.append('')
len(L1_threshold_ceil_hex)

for i in range(112):
  L1_threshold_ceil_hex[i] = deci2hex(str(int(L1_threshold_ceil[i])))


for i in range(112):
  print(i,"     :   ",L1_threshold_ceil_hex[i]+";")

0      :    2;
1      :    2;
2      :    2;
3      :    2;
4      :    2;
5      :    2;
6      :    2;
7      :    2;
8      :    2;
9      :    2;
10      :    2;
11      :    2;
12      :    2;
13      :    2;
14      :    2;
15      :    2;
16      :    2;
17      :    2;
18      :    2;
19      :    2;
20      :    2;
21      :    2;
22      :    2;
23      :    2;
24      :    2;
25      :    2;
26      :    2;
27      :    2;
28      :    2;
29      :    2;
30      :    2;
31      :    2;
32      :    2;
33      :    2;
34      :    2;
35      :    2;
36      :    2;
37      :    2;
38      :    2;
39      :    2;
40      :    2;
41      :    2;
42      :    2;
43      :    2;
44      :    2;
45      :    2;
46      :    2;
47      :    2;
48      :    2;
49      :    2;
50      :    2;
51      :    2;
52      :    2;
53      :    2;
54      :    2;
55      :    2;
56      :    2;
57      :    2;
58      :    2;
59      :    2;
60      :    2;
61      :    2;
62      :    2;
63

In [163]:
# Batchnorm 2
bn2_weight = best_model.bn_L2.weight.data
bn2_weight = bn2_weight.cpu()
bn2_weight = bn2_weight.numpy()

bn2_bias = best_model.bn_L2.bias.data
bn2_bias = bn2_bias.cpu()
bn2_bias = bn2_bias.numpy()

bn2_run_mean = best_model.bn_L2.running_mean.data
bn2_run_mean = bn2_run_mean.cpu()
bn2_run_mean = bn2_run_mean.numpy()

bn2_run_var = best_model.bn_L2.running_var.data
bn2_run_var = bn2_run_var.cpu()
bn2_run_var = bn2_run_var.numpy()

L2_threshold = -1*(bn2_bias*np.sqrt(bn2_run_var)/bn2_weight) + bn2_run_mean

L2_threshold_ceil = np.ceil(L2_threshold)

L2_threshold_ceil_hex = ['']
for i in range(111):
  L2_threshold_ceil_hex.append('')

for i in range(112):
  L2_threshold_ceil_hex[i] = deci2hex(str(int(L2_threshold_ceil[i])))

for i in range(112):
  print(i+112,"     :   ",L2_threshold_ceil_hex[i]+";")

112      :    10;
113      :    1E;
114      :    32;
115      :    F;
116      :    39;
117      :    41;
118      :    16;
119      :    30;
120      :    32;
121      :    19;
122      :    19;
123      :    2C;
124      :    1B;
125      :    3D;
126      :    3E;
127      :    1E;
128      :    17;
129      :    2D;
130      :    1A;
131      :    21;
132      :    30;
133      :    25;
134      :    20;
135      :    30;
136      :    40;
137      :    1C;
138      :    24;
139      :    2E;
140      :    13;
141      :    4C;
142      :    3B;
143      :    33;
144      :    18;
145      :    34;
146      :    19;
147      :    1F;
148      :    27;
149      :    2B;
150      :    33;
151      :    28;
152      :    2E;
153      :    27;
154      :    20;
155      :    21;
156      :    21;
157      :    37;
158      :    2C;
159      :    2F;
160      :    3E;
161      :    39;
162      :    33;
163      :    23;
164      :    52;
165      :    3C;
166      :    38;
167      : 

In [164]:
# Batchnorm 3
bn3_weight = best_model.bn_L3.weight.data
bn3_weight = bn3_weight.cpu()
bn3_weight = bn3_weight.numpy()

bn3_bias = best_model.bn_L3.bias.data
bn3_bias = bn3_bias.cpu()
bn3_bias = bn3_bias.numpy()

bn3_run_mean = best_model.bn_L3.running_mean.data
bn3_run_mean = bn3_run_mean.cpu()
bn3_run_mean = bn3_run_mean.numpy()

bn3_run_var = best_model.bn_L3.running_var.data
bn3_run_var = bn3_run_var.cpu()
bn3_run_var = bn3_run_var.numpy()

L3_threshold = -1*(bn3_bias*np.sqrt(bn3_run_var)/bn3_weight) + bn3_run_mean

L3_threshold_ceil = np.ceil(L3_threshold)

L3_threshold_ceil_hex = ['']
for i in range(111):
  L3_threshold_ceil_hex.append('')

for i in range(112):
  L3_threshold_ceil_hex[i] = deci2hex(str(int(L3_threshold_ceil[i])))

for i in range(112):
  print(i+224,"     :   ",L3_threshold_ceil_hex[i]+";")

224      :    43;
225      :    37;
226      :    2F;
227      :    2F;
228      :    43;
229      :    39;
230      :    2D;
231      :    26;
232      :    29;
233      :    3B;
234      :    2D;
235      :    45;
236      :    34;
237      :    3D;
238      :    2F;
239      :    28;
240      :    2C;
241      :    27;
242      :    32;
243      :    34;
244      :    32;
245      :    52;
246      :    4B;
247      :    34;
248      :    37;
249      :    53;
250      :    3F;
251      :    2A;
252      :    47;
253      :    2B;
254      :    30;
255      :    42;
256      :    20;
257      :    4A;
258      :    2C;
259      :    54;
260      :    5F;
261      :    2C;
262      :    3B;
263      :    20;
264      :    44;
265      :    37;
266      :    20;
267      :    31;
268      :    52;
269      :    38;
270      :    43;
271      :    31;
272      :    1E;
273      :    23;
274      :    1F;
275      :    47;
276      :    39;
277      :    31;
278      :    44;
279      :

In [11]:
def Binarize(tensor,quant_mode='det'): # sign function
    if quant_mode=='det':
        tensor = tensor.sign()
        neg = torch.ones_like(tensor)
        neg = torch.neg(neg)
        tensor = torch.where(tensor == 0, neg, tensor)
        return tensor

In [165]:
best_test_loader1 = torch.load('./save_best/final/test_loader_point7.pt')
best_test_acc = test(best_test_loader1, best_model, criterion1)

Test Error: 
 Accuracy_human: 90.0%, Avg loss: 0.063761 



In [166]:
images, labels = next(iter(best_test_loader1))
images.shape, labels.shape


(torch.Size([32, 1, 36, 28]), torch.Size([32]))

In [179]:
images[6], labels[6], images.shape

(tensor([[[-0.2654, -0.2114, -0.2654,  ...,  1.6911,  0.2967,  1.2046],
          [-0.2546, -0.2222, -0.2330,  ...,  1.3452,  0.4912,  1.2263],
          [-0.2654, -0.2114, -0.2438,  ...,  1.6478,  0.2318,  1.5073],
          ...,
          [-0.2546, -0.2438, -0.2654,  ...,  1.8424,  1.9829,  2.4153],
          [-0.2546, -0.2330, -0.2438,  ...,  1.6262,  0.3723,  0.3831],
          [-0.2438, -0.2438, -0.2546,  ...,  0.4804,  0.9668,  0.1561]]]),
 tensor(9),
 torch.Size([32, 1, 36, 28]))

In [183]:
image = images[6]
images[6], labels[6]

(tensor([[[-0.2654, -0.2114, -0.2654,  ...,  1.6911,  0.2967,  1.2046],
          [-0.2546, -0.2222, -0.2330,  ...,  1.3452,  0.4912,  1.2263],
          [-0.2654, -0.2114, -0.2438,  ...,  1.6478,  0.2318,  1.5073],
          ...,
          [-0.2546, -0.2438, -0.2654,  ...,  1.8424,  1.9829,  2.4153],
          [-0.2546, -0.2330, -0.2438,  ...,  1.6262,  0.3723,  0.3831],
          [-0.2438, -0.2438, -0.2546,  ...,  0.4804,  0.9668,  0.1561]]]),
 tensor(9))

In [184]:
image_bi = Binarize(image)
image_bi

tensor([[[-1., -1., -1.,  ...,  1.,  1.,  1.],
         [-1., -1., -1.,  ...,  1.,  1.,  1.],
         [-1., -1., -1.,  ...,  1.,  1.,  1.],
         ...,
         [-1., -1., -1.,  ...,  1.,  1.,  1.],
         [-1., -1., -1.,  ...,  1.,  1.,  1.],
         [-1., -1., -1.,  ...,  1.,  1.,  1.]]])

In [182]:
result = best_model(torch.reshape(image,(1,1,36,28)).to(device))
result # class 12에 해당하는 각 결과값

tensor([[  4., -28.,   8.,  18.,  -8.,   4.,  46.,  12., -10.,  64., -20., -16.]],
       device='cuda:0', grad_fn=<MmBackward0>)